In [4]:
import pandas as pd
from pathlib import Path

data_path = Path("..") / "Data" / "Raw" / "citibike-2022"
all_files = sorted(data_path.glob("*.csv"))

dfs = []
for file in all_files:
    df_month = pd.read_csv(file)
    dfs.append(df_month)

citibike_2022 = pd.concat(dfs, ignore_index=True)

In [5]:
citibike_2022["started_at"] = pd.to_datetime(citibike_2022["started_at"])
citibike_2022["ended_at"] = pd.to_datetime(citibike_2022["ended_at"])

citibike_2022["date"] = citibike_2022["started_at"].dt.date
citibike_2022["date"] = pd.to_datetime(citibike_2022["date"])
citibike_2022["day_of_week"] = citibike_2022["started_at"].dt.day_name()
citibike_2022["hour"] = citibike_2022["started_at"].dt.hour

citibike_2022["trip_duration_minutes"] = (
    citibike_2022["ended_at"] - citibike_2022["started_at"]
).dt.total_seconds() / 60

In [6]:
daily_trips = (
    citibike_2022
    .groupby("date")
    .agg(
        daily_trips=("ride_id", "count"),           # or the appropriate ID column name
        avg_trip_duration_min=("trip_duration_minutes", "mean")
    )
    .reset_index()
)

In [7]:
citibike_2022.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'day_of_week', 'hour',
       'trip_duration_minutes'],
      dtype='object')

In [8]:
daily_trips.info()
daily_trips.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   365 non-null    datetime64[ns]
 1   daily_trips            365 non-null    int64         
 2   avg_trip_duration_min  365 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 8.7 KB


,date,daily_trips,avg_trip_duration_min
0,2022-01-01,592,42.144369
1,2022-01-02,1248,27.255155
2,2022-01-03,832,24.766146
3,2022-01-04,934,17.051035
4,2022-01-05,914,9.903556


In [9]:
citibike_2022["started_at"] = pd.to_datetime(citibike_2022["started_at"])
citibike_2022["date"] = citibike_2022["started_at"].dt.date
citibike_2022["date"] = pd.to_datetime(citibike_2022["date"])
citibike_2022["day_of_week"] = citibike_2022["started_at"].dt.day_name()
citibike_2022["hour"] = citibike_2022["started_at"].dt.hour

# Duration if not already available
citibike_2022["trip_duration_minutes"] = (
    pd.to_datetime(citibike_2022["ended_at"]) - citibike_2022["started_at"]
).dt.total_seconds() / 60

In [10]:
daily_trips = (
    citibike_2022
    .groupby("date")
    .agg(
        daily_trips=("ride_id", "count"),  # adjust to correct ID column
        avg_trip_duration_min=("trip_duration_minutes", "mean")
    )
    .reset_index()
)
daily_trips.head()

,date,daily_trips,avg_trip_duration_min
0,2022-01-01,592,42.144369
1,2022-01-02,1248,27.255155
2,2022-01-03,832,24.766146
3,2022-01-04,934,17.051035
4,2022-01-05,914,9.903556


This block loops through all monthly CitiBike CSV files in the 2022 folder,
reads each file into a temporary DataFrame, and appends it to a list.
After loading all individual months, pd.concat() combines them into a single
DataFrame (citibike_2022) containing all CitiBike trips for 2022.

### For daily aggregation:
Here we convert the ride start timestamp into a calendar date and use it to
group the data at a daily level. For each day we calculate the total number of
trips (daily_trips) and the average trip duration in minutes. The result is a
compact daily data set that we can later merge with the daily weather data.

In [11]:
import requests
import pandas as pd
from pathlib import Path

# 1) Your personal NOAA token (paste the real one between the quotes)
TOKEN = "SkEXHUNjuoevpJHdLryZbpKCcTXkQGQy"

# 2) API endpoint
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"

# 3) Request parameters: LaGuardia daily data for 2022
params = {
    "datasetid": "GHCND",              
    "stationid": "GHCND:USW00014732",     
    "startdate": "2022-01-01",
    "enddate": "2022-12-31",
    "datatypeid": ["TMAX", "TMIN", "TAVG", "PRCP"],
    "units": "metric",
    "limit": 1000                       
}

headers = {"token": TOKEN}

response = requests.get(base_url, headers=headers, params=params)
response.raise_for_status() 

data = response.json()["results"]
len(data)

1000

In [12]:
weather_long = pd.DataFrame(data)
weather_long.head()
weather_long.columns

Index(['date', 'datatype', 'station', 'attributes', 'value'], dtype='object')

In [13]:
weather_long = weather_long[["date", "datatype", "value"]]

# Pivot: rows = date, columns = datatype (TMAX, TMIN, TAVG, PRCP)
weather = weather_long.pivot_table(
    index="date",
    columns="datatype",
    values="value",
    aggfunc="first"
).reset_index()

# Flatten the column index
weather.columns.name = None

# Rename columns to nicer names
weather = weather.rename(
    columns={
        "date": "date",
        "TAVG": "temp_avg_c",
        "TMAX": "temp_max_c",
        "TMIN": "temp_min_c",
        "PRCP": "precip_mm",
    }
)

# Convert date to datetime
weather["date"] = pd.to_datetime(weather["date"]).dt.date
weather["date"] = pd.to_datetime(weather["date"])

weather.head()

,date,precip_mm,temp_avg_c,temp_max_c,temp_min_c
0,2022-01-01,19.3,11.6,13.9,10.0
1,2022-01-02,1.0,11.4,15.6,3.9
2,2022-01-03,0.0,1.4,3.9,-4.3
3,2022-01-04,0.0,-2.7,2.2,-6.0
4,2022-01-05,6.1,3.2,8.9,0.0


In [14]:
output_path = Path("..") / "Data" / "Raw" / "weather_lga_2022.csv"
weather.to_csv(output_path, index=False)
output_path

WindowsPath('../Data/Raw/weather_lga_2022.csv')

In [15]:
citibike_weather_daily = pd.merge(
    daily_trips,
    weather,
    on="date",
    how="left"
)

citibike_weather_daily.head()

,date,daily_trips,avg_trip_duration_min,precip_mm,temp_avg_c,temp_max_c,temp_min_c
0,2022-01-01,592,42.144369,19.3,11.6,13.9,10.0
1,2022-01-02,1248,27.255155,1.0,11.4,15.6,3.9
2,2022-01-03,832,24.766146,0.0,1.4,3.9,-4.3
3,2022-01-04,934,17.051035,0.0,-2.7,2.2,-6.0
4,2022-01-05,914,9.903556,6.1,3.2,8.9,0.0


In [16]:
processed_path = Path("..") / "Data" / "Processed" / "citibike_weather_daily_2022.csv"
citibike_weather_daily.to_csv(processed_path, index=False)

In [17]:
citibike_2022.info()
citibike_2022.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895485 entries, 0 to 895484
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   ride_id                895485 non-null  object        
 1   rideable_type          895485 non-null  object        
 2   started_at             895485 non-null  datetime64[ns]
 3   ended_at               895485 non-null  datetime64[ns]
 4   start_station_name     895475 non-null  object        
 5   start_station_id       895475 non-null  object        
 6   end_station_name       892281 non-null  object        
 7   end_station_id         892281 non-null  object        
 8   start_lat              895485 non-null  float64       
 9   start_lng              895485 non-null  float64       
 10  end_lat                893515 non-null  float64       
 11  end_lng                893515 non-null  float64       
 12  member_casual          895485 non-null  obje

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,day_of_week,hour,trip_duration_minutes
0,CA5837152804D4B5,electric_bike,2022-01-26 18:50:39,2022-01-26 18:51:53,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-26,Wednesday,18,1.233333
1,BA06A5E45B6601D2,classic_bike,2022-01-28 13:14:07,2022-01-28 13:20:23,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member,2022-01-28,Friday,13,6.266667
2,7B6827D7B9508D93,classic_bike,2022-01-10 19:55:13,2022-01-10 20:00:37,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member,2022-01-10,Monday,19,5.400000
3,6E5864EA6FCEC90D,electric_bike,2022-01-26 07:54:57,2022-01-26 07:55:22,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-26,Wednesday,7,0.416667
4,E24954255BBDE32D,electric_bike,2022-01-13 18:44:46,2022-01-13 18:45:43,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-13,Thursday,18,0.950000


In [26]:
import pandas as pd

# Ensure 'date' is datetime in both
daily_trips["date"] = pd.to_datetime(daily_trips["date"])
weather_reset = weather.reset_index().rename(columns={"index": "date"})
weather_reset["date"] = pd.to_datetime(weather_reset["date"])

final_df = pd.merge(
    daily_trips,
    weather_reset,
    on="date",
    how="inner"
)

final_df.to_csv(r"Data\Processed\citibike_weather_merged_2022.csv", index=False)
final_df.head()

ValueError: cannot insert date, already exists

In [28]:
import pandas as pd

# 1) Clean daily_trips date
daily_trips = daily_trips.copy()
daily_trips["date"] = pd.to_datetime(daily_trips["date"])

# 2) Create weather_reset with ONE date column
weather_reset = weather.copy()

# If the index is datetime, use it as the date column
weather_reset.index = pd.to_datetime(weather_reset.index)
weather_reset = weather_reset.reset_index().rename(columns={"index": "date"})

# If weather also has a 'date' column already, drop the duplicate
if "date" in weather.columns:
    # after reset_index(), the original 'date' column (if it existed) will still be there,
    # so remove it to keep only the index-derived date
    weather_reset = weather_reset.loc[:, ~weather_reset.columns.duplicated()]
    # If there are still two date-like columns, keep the first 'date' and drop the rest
    extra_date_cols = [c for c in weather_reset.columns if c.lower() == "date"][1:]
    if extra_date_cols:
        weather_reset = weather_reset.drop(columns=extra_date_cols)

# Ensure weather_reset['date'] is datetime
weather_reset["date"] = pd.to_datetime(weather_reset["date"])

# 3) Merge
final_df = pd.merge(daily_trips, weather_reset, on="date", how="inner")

# 4) Save
final_df.to_csv(r"Data\Processed\citibike_weather_merged_2022.csv", index=False)

final_df.head()

ValueError: cannot insert date, already exists

In [30]:
import pandas as pd

# daily_trips: ensure date index
daily_trips2 = daily_trips.copy()
daily_trips2["date"] = pd.to_datetime(daily_trips2["date"])
daily_trips2 = daily_trips2.set_index("date").sort_index()

# weather: ensure datetime index
weather2 = weather.copy()

# If weather has a duplicate 'date' column, drop it (keep the index as the date source of truth)
if "date" in weather2.columns:
    weather2 = weather2.drop(columns=["date"])

weather2.index = pd.to_datetime(weather2.index)
weather2 = weather2.sort_index()
weather2.index.name = "date"

# Join on index (now safe)
final_df = daily_trips2.join(weather2, how="inner")

final_df.to_csv(r"C:\Users\User\Downloads\Citibike-Weather-Dashboard\Data\Processed\citibike_weather_merged_2022.csv", index=True)
final_df.head()

,daily_trips,avg_trip_duration_min,precip_mm,temp_avg_c,temp_max_c,temp_min_c
date,,,,,,
